In [1]:
customers = spark.read.option("header",True).csv("Customers.csv")
customers.show()

+---+-------+
| ID|   Name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
+---+-------+



In [2]:
from pyspark.sql.functions import col
filtered_cust = customers.where(col("Name").isin(['Alice', 'Charlie']))
filtered_cust.show()

+---+-------+
| ID|   Name|
+---+-------+
|  1|  Alice|
|  3|Charlie|
+---+-------+



In [3]:
orders = spark.read.option("header",True).csv("Orders.csv").withColumnRenamed("ID", "order_ID")
orders.show()

+--------+--------+-----+
|order_ID|Customer|Value|
+--------+--------+-----+
|       1|       1|   14|
|       2|       2|    2|
|       3|       1|   21|
|       4|       3|    5|
|       5|       3|    9|
|       6|       3|   25|
+--------+--------+-----+



In [4]:
joined = filtered_cust.join(orders, col("ID") == col("Customer"))
joined.show()

+---+-------+--------+--------+-----+
| ID|   Name|order_ID|Customer|Value|
+---+-------+--------+--------+-----+
|  1|  Alice|       1|       1|   14|
|  1|  Alice|       3|       1|   21|
|  3|Charlie|       4|       3|    5|
|  3|Charlie|       5|       3|    9|
|  3|Charlie|       6|       3|   25|
+---+-------+--------+--------+-----+



In [5]:
joined.select('Name', 'Customer', 'Value').show()

+-------+--------+-----+
|   Name|Customer|Value|
+-------+--------+-----+
|  Alice|       1|   14|
|  Alice|       1|   21|
|Charlie|       3|    5|
|Charlie|       3|    9|
|Charlie|       3|   25|
+-------+--------+-----+



In [6]:
joined.groupBy('ID').agg({'Value': 'sum'}).show()

+---+----------+
| ID|sum(Value)|
+---+----------+
|  3|      39.0|
|  1|      35.0|
+---+----------+



In [7]:
customers.createOrReplaceTempView("customers")
orders.createOrReplaceTempView("orders")

In [8]:
spark.sql('select * from customers').show()

+---+-------+
| ID|   Name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
+---+-------+



In [9]:
spark.sql(
'''
select sum(Value) as Total_Value 
from customers join orders 
on ID=Customer 
where Name in ("Alice", "Charlie") 
group by ID
'''
).show()

+-----------+
|Total_Value|
+-----------+
|       39.0|
|       35.0|
+-----------+

